In [1]:
from torch.nn.modules.activation import ReLU

import torch
import torch.nn as nn

In [2]:
def conv_bn_relu(in_channels, out_channels, kernel_size=3, stride=1, padding=0):
    """
    합성곱 연산, Batch Normalization, ReLU 활성함수를 연속적으로 거치도록 하는 Sequential을 return하는 함수

    매개변수(Parameters)
    ----------------------
    in_channels: int형, 입력으로 들어오는 이미지의 채널 개수
    out_channels: int형, 출력으로 반환할 이미지의 채널 개수
    kernel_size: int 혹은 tuple 형, 사용할 필터의 크기 정보, default 값 3
    stride: int 혹은 tuple 형, 스트라이드 값, defalut 값 1
    padding: int 혹은 tuple 혹은 str형, 패딩에 대한 정보, defalut 값 0(패딩 없음)

    반환 값(Returns)
    ----------------------
    합성곱 연산, Batch Normalization, ReLU 활성함수를 연속적으로 거치도록 하는 torch.nn.Sequential
    """
    return nn.Sequential(
        # 입력받은 매개변수에 따라, 합성곱 연산을 진행
         nn.Conv2d(in_channels,out_channels,kernel_size=kernel_size,
                  stride=stride,padding=padding,bias=False),
        # 배치 정규화 진행
         nn.BatchNorm2d(out_channels) ,
        # 활성함수인 ReLU 함수 거치기
         nn.ReLU(inplace=True) 
    )

In [3]:
def SeparableConv2D(in_channels, out_channels, kernel=3):
    """
    Separable Convolution 연산을 진행하는 Sequential을 return하는 함수

    매개변수(Parameters)
    ----------------------
    in_channels: int형, 입력으로 들어오는 이미지의 채널 개수
    out_channels: int형, 출력으로 반환할 이미지의 채널 개수
    kernel: int 혹은 tuple 형, 사용할 필터의 크기 정보, default 값 3
    
    반환 값(Returns)
    ----------------------
    Separable Convolution 연산을 진행하는 Sequential
    """
    return nn.Sequential(
        # 입력받은 채널의 개수를 보존하는 형태로, 채널 수가 1인 필터를 합성곱 연산
         nn.Conv2d(in_channels,in_channels,kernel_size=kernel,
                  stride=1,groups=in_channels,padding=1,bias=False),
        # 1x1 합성곱 연산 진행, 채널 개수를 원하는 출력 채널 개수로 조정
         nn.Conv2d(in_channels,out_channels,kernel_size=1,
                  stride=1,bias=False),
    )

In [4]:
class ResidualXceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel=3):
        super(ResidualXceptionBlock, self).__init__()


        self.depthwise_conv1 = SeparableConv2D(in_channels, out_channels, kernel)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU(inplace=True)

        self.depthwise_conv2 = SeparableConv2D(out_channels, out_channels, kernel)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # self.padd = nn.ZeroPad2d(22)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.residual_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2, padding=22, bias=False)
        self.residual_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.residual_bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        # residual branch
        residual = self.residual_conv(x)
        residual = self.residual_bn(residual)
        
        # print('input',x.shape)
        # feature extraction branch
        x = self.depthwise_conv1(x)
        # print('conv1',x.shape)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.depthwise_conv2(x)
        x = self.bn2(x)
        # print('conv2',x.shape)

        # x = self.padd(x)
        x = self.maxpool(x)
        # print(x[:,:, 11:22, 11:22])
        # print('max_pooling',x.shape)
        # print('res',residual.shape)
        return x + residual

In [5]:
class Mini_Xception(nn.Module):
    def __init__(self):
        super(Mini_Xception, self).__init__()

        # self.conv1 = conv_bn_relu(1, 32, kernel_size=3, stride=1, padding=0)
        # self.conv2 = conv_bn_relu(32, 64, kernel_size=3, stride=1, padding=0)
        # self.residual_blocks = nn.ModuleList([
        #     ResidualXceptionBlock(64 , 128).to(device),
        #     ResidualXceptionBlock(128, 256).to(device),
        #     ResidualXceptionBlock(256, 512).to(device),
        #     ResidualXceptionBlock(512, 1024).to(device)            
        # ])

        # self.conv3 = nn.Conv2d(1024, 7, kernel_size=3, stride=1, padding=1)


        self.conv1 = conv_bn_relu(1, 8, kernel_size=3, stride=1, padding=0)
        self.conv2 = conv_bn_relu(8, 8, kernel_size=3, stride=1, padding=0)
        self.residual_blocks = nn.ModuleList([
            ResidualXceptionBlock(8 , 16),
            ResidualXceptionBlock(16, 32),
            ResidualXceptionBlock(32, 64),
            ResidualXceptionBlock(64, 128)            
        ])
        self.conv3 = nn.Conv2d(128, 7, kernel_size=3, stride=1, padding=1)

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.conv2(x)

        for block in self.residual_blocks:
            x = block(x)
            # print('ith block', x.shape, block.device)

        # print('blocks:',x.shape)
        x = self.conv3(x)
        
        # print('conv3',x.shape)
        x = self.global_avg_pool(x)
        # # x = self.softmax(x)
        
        return x

In [6]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout2d(0.1)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout2d(0.1)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout2d(0.1)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout2d(0.1)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
            nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout2d(0.1)
        )
        self.flatten=nn.Flatten()
        
        self.fc1 = nn.Linear(512*1*1,128)
        self.fc2 = nn.Linear(128,128)
        self.fc3=nn.Linear(128,7)
        
    def forward(self, x):
        #여기에 코드 작성
        x=self.conv1(x)

        x=self.conv2(x)

        x=self.conv3(x)

        x=self.conv4(x)
        x=self.conv5(x)
        x=self.flatten(x)

        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fc3(x)
        
        return x

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=7):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv1 = nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


class ResNet_AE(nn.Module):
    def __init__(self, block, num_blocks, num_classes=7):
        super(ResNet_AE, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 1, kernel_size=3,stride=1, padding=1, bias=False),
            nn.Sigmoid()
        )

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)


    def forward(self, x):
        out1 = self.conv1(x)
        
        decoded = self.decoder(out1)

        out = F.relu(self.bn1(out1))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out, x, decoded


def ResNet18_AE():
    return ResNet_AE(BasicBlock, [2, 2, 2, 2])


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [8]:
!pip install torchinfo
from torchinfo import summary

In [9]:
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
    model1 = NN()
    model2=Mini_Xception()
    model3=ResNet50()
    print(summary(model1,(128,1,48,48)))
    print(summary(model2,(128,1,48,48)))
    print(summary(model3,(32,1,48,48)))


Layer (type:depth-idx)                   Output Shape              Param #
NN                                       [128, 7]                  --
├─Sequential: 1-1                        [128, 64, 24, 24]         --
│    └─Conv2d: 2-1                       [128, 64, 48, 48]         640
│    └─Conv2d: 2-2                       [128, 64, 48, 48]         36,928
│    └─BatchNorm2d: 2-3                  [128, 64, 48, 48]         128
│    └─ReLU: 2-4                         [128, 64, 48, 48]         --
│    └─MaxPool2d: 2-5                    [128, 64, 24, 24]         --
│    └─Dropout2d: 2-6                    [128, 64, 24, 24]         --
├─Sequential: 1-2                        [128, 128, 12, 12]        --
│    └─Conv2d: 2-7                       [128, 128, 24, 24]        73,856
│    └─Conv2d: 2-8                       [128, 128, 24, 24]        147,584
│    └─BatchNorm2d: 2-9                  [128, 128, 24, 24]        256
│    └─ReLU: 2-10                        [128, 128, 24, 24]        --